In [ ]:
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

#현재 작업 위치 이동
#띄어쓰기에 \붙일 것
%cd /content/drive/MyDrive/멀티캠퍼스 파이썬/최종 프로젝트

Mounted at /content/drive
/content/drive/MyDrive/멀티캠퍼스 파이썬/최종 프로젝트


In [ ]:
# 패키지 설치

import tensorflow as tf
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.models import load_model
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
# import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import nltk
import joblib
import pickle
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# 메타 전문가 리뷰

critic_data = pd.read_csv('./data/meta_final/meta_meta_mart.csv')

# game / review 열로 가져오기, 조건 설정
d1 = critic_data[(critic_data['platform']=='pc') & (critic_data['criticscore']>=75)][['appid', 'criticcontent']]

# 이름 각각 game, score으로 변경
d1.rename(columns = {'appid':'game', 'criticcontent':'review'}, inplace = True)

# 리뷰 NaN값인 행 제거
d1.dropna(subset=['review'], inplace=True) 



# 메타 유저 리뷰

user_data = pd.read_csv("./data/meta_final/meta_user_mart.csv")

# game / review 열로 가져오기, 조건 설정
d2 = user_data[(user_data['platform']=='pc') & (user_data['userscore']>7.5)][['appid', 'usercontent']]

# 이름 각각 user, game, score으로 변경
d2.rename(columns = {'appid':'game', 'usercontent':'review'}, inplace = True)

# 리뷰 NaN값인 행 제거
d2.dropna(subset=['review'], inplace=True) 




# 스팀 리뷰 데이터

steam_review = pd.read_csv("./data/steam_data/for_mimi.csv")

steam_review.dropna(subset=['review'], inplace=True) # 리뷰 NaN값인 행 제거
steam_review.dropna(subset=['weighted_vote_score'], inplace=True) # weighted_vote_score NaN값인 행 제거
steam_review.dropna(subset=['voted_up'], inplace=True) # voted_up NaN값인 행 제거
steam_review.dropna(subset=['appid'], inplace=True) # appid NaN값인 행 제거

d3 = steam_review[(steam_review['voted_up']==True) & (steam_review['weighted_vote_score']>=0.75)][['appid', 'review']]

# 이름 각각 game, score으로 변경
d3.rename(columns = {'appid':'game'}, inplace = True)




# d1, d2, d3 합치기
total_data = pd.concat([d1, d2, d3])
# nan값 있는 행 제거
total_data.dropna(inplace=True)
# 게임 명 공백인 행 제거
drop_index = total_data[total_data['game']==''].index
total_data.drop(drop_index, axis='index', inplace=True)
# game열 타입 float에서 int로
total_data = total_data.astype({'game':'int'})

total_data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,game,review
0,632470.0,Traversing the mind of your character is as mu...
1,632470.0,Disco Elysium – The Final Cut is something rar...
2,632470.0,"Enhanced with fantastic and fresh content, Dis..."
3,632470.0,A fully voiced cast and new content of compara...
4,632470.0,Disco Elysium: The Final Cut doesn't shy away ...


In [ ]:

total_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185878 entries, 0 to 11972685
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   game    185878 non-null  int64 
 1   review  185878 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.3+ MB


In [ ]:
total_data.head()

,game,review
0,632470,Traversing the mind of your character is as mu...
1,632470,Disco Elysium – The Final Cut is something rar...
2,632470,"Enhanced with fantastic and fresh content, Dis..."
3,632470,A fully voiced cast and new content of compara...
4,632470,Disco Elysium: The Final Cut doesn't shy away ...


In [ ]:
# 희귀단어 기준 계산
# tokenizer로 total_data['review'] 단어사전 만들고 희귀단어 비율 보기

tok = Tokenizer() # 토크나이저 할당
tok.fit_on_texts(total_data['review']) # 리뷰 데이터로 토크나이저 학습

# 등장 빈도수가 3회 이하 단어를 희귀단어로 취급
threshold = 3
total_cnt = len(tok.word_index) # 토크나이저가 생성한 단어사전의 단어 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합

for key, value in tok.word_counts.items():
    total_freq = total_freq + value
    if(value <= threshold):
        rare_cnt = rare_cnt + 1

# 희귀단어 기준
vocab_size = total_cnt- rare_cnt + 1 

In [ ]:
vocab_size

50316

In [ ]:
# 전처리
def remain_engnum(sentence):
  sentence = re.sub('[^a-zA-Z0-9]+', ' ', sentence)
  return sentence

tqdm.pandas()
# 영어, 숫자만 빼고 제거
total_data['review'] = total_data['review'].progress_apply(lambda x:remain_engnum(x))

# 소문자화
total_data['review']= total_data['review'].apply(lambda x: x.lower())

# 게임 별로 리뷰 합치기
game_review = pd.DataFrame(total_data['game'].unique(), columns=['game'])

for game in tqdm(total_data['game'].unique()):
  game_review.loc[game_review['game']==game, 'review'] = " ".join(total_data[total_data['game']==game]['review'])

# 희귀단어 제거를 위한 토크나이저 생성
total_tokenizer = Tokenizer(num_words = vocab_size) # 단어사전 생성
total_tokenizer.fit_on_texts(game_review['review']) # 학습

# 단어사전에서 인덱스가 71297 초과인 단어(=희귀단어) 제거
words_frequency = [word for word, index in total_tokenizer.word_index.items() if index >= vocab_size + 1]
for word in words_frequency:
    del total_tokenizer.word_index[word] # 해당 단어에 대한 인덱스 정보를 삭제
    del total_tokenizer.word_counts[word] # 해당 단어에 대한 카운트 정보를 삭제

# 토큰화
game_review['review'] = game_review['review'].progress_apply(lambda x:text_to_word_sequence(x))

# 희귀단어 제거 함수
def remain_word(review):
  result = []
  for word in review:
    if word in total_tokenizer.word_index.keys():
      result.append(word)
  return result

game_review['review'] = game_review['review'].progress_apply(lambda x:remain_word(x))

# 불용어 제거
stop_words = set(stopwords.words('english'))

def remove_stopwords(text): 
    output= [i for i in text if i not in stop_words] 
    return output

game_review['review'] = game_review['review'].progress_apply(lambda x:remove_stopwords(x))

100%|██████████| 7939/7939 [00:01<00:00, 4999.30it/s]


In [ ]:
game_review.head()

,game,review
0,632470,"[traversing, mind, character, much, plot, case..."
1,220,"[nothing, less, revolution, gaming, industry, ..."
2,70,"[everyday, come, across, game, totally, revolu..."
3,620,"[portal, 2, masterpiece, work, art, love, inge..."
4,1245620,"[elden, ring, walking, example, 10, 10, game, ..."


In [ ]:
# 데이터에 태그 달기

doc_df = game_review[['game','review']].values.tolist()
tagged_data = [TaggedDocument(words=_d, tags=[gid]) for gid, _d in doc_df]

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [ ]:
tagged_data[0]

In [ ]:
model = Doc2Vec(
    window=5, # 모델 학습 시 앞 뒤로 보는 단어 개수
    size=100, # 벡터차원 크기
    alpha=0.025, # learning rate
    min_alpha=0.025,# min learning rate. 학습 진행될수록 alpha값이 min_alpha로 근사
    min_count=3, # 학습에 사용할 최소 단어 빈도 수
    dm =1, # 학습 방법 : 1-pvdm, 0-pvdbow
    negative = 5, # complexity reudction. negative sampling 시 샘플링 할 단어 개수 지정
    seed = 9999)

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [ ]:
with open('./data/appidtagdata.pkl', 'wb') as f: # wb : 피클 작성 시 / rb : 피클 열람 시
	pickle.dump(tagged_data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
model.build_vocab(tagged_data)

max_epochs = 10

for epoch in range(max_epochs): #10번 학습
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save('./model/appiddoc2vec.model')

iteration 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if __name__ == '__main__':


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [ ]:
# 추천 게임 출력
def gentleman_ver1(game1, game2, game3):
  d1 = pd.DataFrame(model.docvecs.most_similar([game1], topn=10), columns=['game', 'similarity1']) # game1 추천목록 & 유사도
  d2 = pd.DataFrame(model.docvecs.most_similar([game2], topn=10), columns=['game', 'similarity2']) # game2 추천목록 & 유사도
  d3 = pd.DataFrame(model.docvecs.most_similar([game3], topn=10), columns=['game', 'similarity3']) # game3 추천목록 & 유사도
  df = pd.merge(d1, d2, on='game', how='outer') # d1, d2 merge
  df = pd.merge(df, d3, on='game', how='outer') # d2, d3 merge

  for game in df['game']: # ['game'] 행 하나씩 돌면서 
    if game in [game1, game2, game3]: # game이 game1, game2,m game3에 해당하는 경우
      drop_index = df[df['game']==game].index
      df.drop(drop_index, axis='index', inplace=True) # 해당 행 drop

  # 추천 리스트 간 중복되는 게임이 없는 경우 각 추천리스트에서 2개씩 뽑음
  if len(df) == 30: 
    return [x[0] for x in model.docvecs.most_similar([game1], topn=2)] + [x[0] for x in model.docvecs.most_similar([game2], topn=2)] + [x[0] for x in model.docvecs.most_similar([game3], topn=2)]
  # 유사도 평균 내서 상위 6개 추출
  else : 
    df.fillna(0, inplace=True)
    df['mean'] = df.mean(axis=1, numeric_only=True)
    df.sort_values('mean', ascending=False, inplace=True)
    return list(df['game'])[:6]

In [ ]:
gentleman_ver1(413150, 304430, 1172470)

[666140, 433340, 424840, 231160, 1399780, 444090]

In [ ]:
info = pd.read_csv("./data/meta_final/mergeinfo.csv")

In [ ]:
info[info['full_name']=='CROSSCORD']['appid']

Series([], Name: appid, dtype: int64)

In [ ]:
info[info['appid']==433340]['title']

18399    Slime Rancher
Name: title, dtype: object